In [1]:
import tweepy

In [2]:
f = open('secret.txt', 'r')
mess = f.read().split('\n')
consumer_key = mess[0]
consumer_secret = mess[1]
access_token = mess[2]
access_token_secret = mess[3]
f.close()

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
import urllib
import datetime
import csv
from bs4 import BeautifulSoup
import time

In [5]:
class Tweet(object):
    def __init__(self, tweet_id, tweet_text):
        self.id = tweet_id
        self.text = tweet_text

In [6]:
def get_tweets(query):
    
    # Craft the query
    params = {
        'f': 'tweets',
        'q': query,
        'lang': 'en',
    }

    q_string = urllib.parse.urlencode(params)
    url = 'https://twitter.com/search?' + q_string
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    req = urllib.request.Request(url, headers=headers)
    resp = urllib.request.urlopen(req)
    
    # Scrape the tweets
    soup = BeautifulSoup(resp.read(), "html.parser")
    tweets = soup.find_all('li','js-stream-item')
    
    tweet_list = []
    for tweet in tweets:
        if tweet.find('p', 'TweetTextSize'):
            tweet_text = tweet.find('p','TweetTextSize').text.encode('utf8')
            tweet_id = tweet['data-item-id']
            tweet_list.append(Tweet(tweet_id, tweet_text))
        else:
            continue
            
    return tweet_list

In [7]:
bernie_search = ['Bernie', 'Sanders', 'BernieSanders']
rubio_search = ['Marco', 'Rubio', 'marcorubio']
cruz_search = ['Ted', 'Cruz', 'tedcruz']
clinton_search = ['Hilary', 'Clinton', 'HilaryClinton']
trump_search = ['Donald', 'Trump', 'DonaldTrump']

bernie_keywords = ['Bernie', 'Sanders', 'BernieSanders', 'Feel the Bern', 'Bern', 'feelthebern']
rubio_keywords = ['Marco', 'Rubio', 'marcorubio', '"A New American Century"']
cruz_keywords = ['Ted', 'Cruz', 'tedcruz', 'CruzCrew', '"Together, we will win"']
clinton_keywords = ['Hilary', 'Clinton', 'HilaryClinton', '"I\'m with her"', '"Stronger Together"', 'imwithher']
trump_keywords = ['Donald', 'Trump', 'DonaldTrump', 'NeverTrump', '"Make America Great Again"', 'maga']

primaries = {'Democrat': [('Clinton', clinton_search), 
                          ('Sanders', bernie_search)],
             'Republican': [('Rubio', rubio_search),
                            ('Cruz', cruz_search),
                            ('Trump', trump_search)]}

In [8]:
# Generate the query string
def geo_time_query_part(latitude, longitude, radius, after, before):
    since = datetime.datetime.strptime(after, '%m/%d/%y')
    until = datetime.datetime.strptime(before, '%m/%d/%y')
    
    geo = ' geocode:%.6f,%.6f,%dkm' % (float(latitude), float(longitude), float(radius))
    time_part = (' since:%s' % since.strftime('%Y-%m-%d')) + (' until:%s' % until.strftime('%Y-%m-%d'))
    
    return time_part + geo
    

In [9]:
# Generate all the time-based and geo-based components from the csv
state_dict = dict()

with open('data/raw_calc_lat_lon.csv', 'r') as csvfile:
    tab = csv.reader(csvfile, delimiter=',')
    for row in tab:
        if row[1] != 'latitude':
            state = row[0]
            latitude = row[1]
            longitude = row[2]
            radius = row[3]
            date = row[4]
            party = row[5]
            after = row[6]
            until = row[7]
            
            if state not in state_dict:
                state_dict[state] = dict()
            
            state_dict[state][party] = geo_time_query_part(latitude, longitude, radius, after, until)
            

In [10]:
# Get all the tweets for a person
qs_list = []
tweets = dict()
for state in state_dict.keys():
    tweets[state] = dict()
    
    for primary in primaries.keys():
        tweets[state][primary] = dict()
        for candidate in primaries[primary]:
            geo_time = state_dict[state][primary]
            query_string = " OR ".join(candidate[1]) + geo_time
            
            time.sleep(1)
            dl_tweets = get_tweets(query_string)
            tweets[state][primary][candidate[0]] = dl_tweets
            print('%s %s %s: Downloaded %d tweets.' % (state, primary, candidate[0], len(dl_tweets)))
            

Alabama Democrat Clinton: Downloaded 19 tweets.
Alabama Democrat Sanders: Downloaded 4 tweets.
Alabama Republican Rubio: Downloaded 20 tweets.
Alabama Republican Cruz: Downloaded 20 tweets.
Alabama Republican Trump: Downloaded 20 tweets.
Alaska Democrat Clinton: Downloaded 0 tweets.
Alaska Democrat Sanders: Downloaded 3 tweets.
Alaska Republican Rubio: Downloaded 1 tweets.
Alaska Republican Cruz: Downloaded 6 tweets.
Alaska Republican Trump: Downloaded 0 tweets.
Arizona Democrat Clinton: Downloaded 20 tweets.
Arizona Democrat Sanders: Downloaded 20 tweets.
Arizona Republican Rubio: Downloaded 4 tweets.
Arizona Republican Cruz: Downloaded 20 tweets.
Arizona Republican Trump: Downloaded 19 tweets.
Arkansas Democrat Clinton: Downloaded 14 tweets.
Arkansas Democrat Sanders: Downloaded 1 tweets.
Arkansas Republican Rubio: Downloaded 14 tweets.
Arkansas Republican Cruz: Downloaded 8 tweets.
Arkansas Republican Trump: Downloaded 5 tweets.
California Democrat Clinton: Downloaded 20 tweets.
Cal

In [11]:
qs_list

[]

704807367163052032
704805365523701760
704768318175969280
704767968635277312
704765140629991424
704764180566556673
704755625264181248
704754716043968512
704733669018173440
704730200202878976
704730198223114240
704729395009863682
704729169456996352
704728873095790593
704711274651713536
704709199809421312
704693208807292928
704677016042401794
704660409974923264
704626504043331588


In [172]:
ts = get_tweets(qs_list[0])    

In [180]:
ts[5].text

b'Maybe his new plan is to be Hillary Clinton\xe2\x80\x99s VP choice. https://twitter.com/ZekeJMiller/status/704397766584303616\xc2\xa0\xe2\x80\xa6'

In [204]:
%matplotlib

/usr/local/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


Using matplotlib backend: MacOSX


In [12]:
len(tweets)

51

In [15]:
for state in tweets.keys():
    total = 0
    for party in tweets[state].keys():
        for candidate in tweets[state][party].keys():
            total += len(tweets[state][party][candidate])
    print(str(total) + ',')

83,
10,
83,
42,
82,
93,
54,
6,
0,
98,
100,
26,
29,
54,
81,
100,
48,
39,
18,
4,
75,
31,
9,
5,
26,
57,
6,
1,
4,
30,
71,
38,
72,
88,
0,
100,
95,
4,
54,
30,
86,
1,
85,
100,
44,
7,
45,
10,
20,
50,
1,


In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [13]:
import nltk

In [14]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [16]:
sid = SentimentIntensityAnalyzer()

In [19]:
print(tweets['Texas']['Republican']['Cruz'][0].text)

b'Stage set & cameras set @tedcruz watch party location: Redneck Country Club.  @DianaZogaFox4 @FOX4 #Fox4Electionspic.twitter.com/Xc3bIpfU29'


In [25]:

sid.polarity_scores(str(tweets['Texas']['Republican']['Cruz'][4].text))


{'compound': -0.4767, 'neg': 0.136, 'neu': 0.864, 'pos': 0.0}

In [34]:
for twt in tweets['New York']['Democrat']['Sanders']:
    print("%0.4f %s" % (sid.polarity_scores(str(twt.text))['compound'], twt.text))

0.0000 b'Civic duty done. Get out and vote. Polls close at 9pm New York. #GOTV #BernieSanders\xe2\x80\xa6 https://www.instagram.com/p/BEZkk16OjSi/\xc2\xa0'
0.4019 b"#voted...and yes #stillsanders #feelthebern #berniesanders...\n\nY'all be ok...I know I will... @\xe2\x80\xa6 https://www.instagram.com/p/BEZkkxfC1dv/\xc2\xa0"
0.0000 b"They didn't give out stickers, but I voted! #FeelTheBern @berniesanders @ Dr Weeks Elementary\xe2\x80\xa6 https://www.instagram.com/p/BEZZuhny0m2/\xc2\xa0"
0.0000 b"Voted! Let's go Bernie! (@ Town Of Maine Court in Maine, NY) https://www.swarmapp.com/c/iHylxipAJNB\xc2\xa0"
0.4902 b'I JUST VOTED FOR BERNIE and my heart is racing, full to the brim with So Much Hope. #feelthebern\xe2\x80\xa6 https://www.instagram.com/p/BEY3qQ7x119/\xc2\xa0'
0.4738 b"Today is the day New York!! Let's show up for @berniesanders like he has for us\xe2\x80\xa6 https://www.instagram.com/p/BEY260fyJWf/\xc2\xa0"
0.0000 b'Bernie toca a nuestras puertas. @ Ithaca, New York https://www.i

In [35]:
import pandas as pd

In [37]:
df = pd.DataFrame(columns=['State', 'Primary', 'Candidate', 't0','t1','t2', 't3', 't4', 't5', 't6', 't7', 't8',
                           't9', 't10', 't11','t12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20'])

In [47]:
i = 0
for _, state in enumerate(tweets):
    for party in tweets[state].keys():
        for candidate in tweets[state][party].keys():
            df.set_value(i, 'State', state)
            df.set_value(i, 'Primary', party)
            df.set_value(i, 'Candidate', candidate)
            for k in range(len(tweets[state][party][candidate])):
                tweet_text = tweets[state][party][candidate][k].text
                score = sid.polarity_scores(str(tweet_text))['compound']
                df.set_value(i, 't' + str(k), score)
            
            i += 1

In [48]:
df

,State,Primary,Candidate,t0,t1,t2,t3,t4,t5,t6,...,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20
0,Alabama,Democrat,Clinton,0,-0.0644,-0.4767,0,0,0,0,...,0,-0.2225,-0.34,0.4404,0,0,0,0.4995,NaN,NaN
1,Alabama,Democrat,Sanders,0,0.6696,0,-0.1779,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,Republican,Rubio,0.6486,0,0,0.34,0.6249,0,-0.4019,...,0.4201,0.8225,-0.09,-0.6369,0.7906,0.2484,0.836,0.5574,0.3182,NaN
3,Alabama,Republican,Cruz,-0.3182,-0.3861,0.1779,0,0.4201,0,-0.09,...,-0.6705,0,0.5719,0.872,0,-0.3818,0.7876,0,0.5719,NaN
4,Alabama,Republican,Trump,0.7297,-0.3612,0,-0.8765,0.4019,0,0.5423,...,0,-0.09,0.2484,-0.2023,0.3612,-0.4215,0,-0.2617,0.3818,NaN
5,Alaska,Democrat,Clinton,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Alaska,Democrat,Sanders,0,0,0.8221,NaN,NaN,NaN,-0.4404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Alaska,Republican,Rubio,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Alaska,Republican,Cruz,0,0,0,0,0,0.4404,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Alaska,Republican,Trump,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df.to_csv('twitter_sentiment_scores.csv')